In [2]:
import os
print(os.getcwd())

C:\Users\trive


In [4]:
os.chdir('C:/Users/trive/Dashboard')

In [6]:
print(os.getcwd())

C:\Users\trive\Dashboard


In [8]:
!pip install dash==2.9.3  # or latest version
!pip install dash_bootstrap_components

ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [9]:
import pandas as pd
import numpy as np

In [10]:
from prophet import Prophet

model = Prophet()

In [14]:
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go
import dash_bootstrap_components as dbc

In [16]:
# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server
app.title = "Energy Consumption Forecast" 

In [18]:
# Load and process data
df = pd.read_csv('C:/Users/trive/Dashboard/Data/AEP_hourly.csv' , parse_dates=['Datetime'], index_col='Datetime')
print("Original data shape:", df.shape)

Original data shape: (121273, 1)


In [20]:
# Resample to daily data
df_daily = df['AEP_MW'].resample('D').mean().dropna().reset_index()
df_daily.columns = ['ds', 'y']  # Prophet requires 'ds' and 'y' column names

In [22]:
print("Daily data preview:")
print(df_daily.head())
print("Daily data shape:", df_daily.shape)

Daily data preview:
          ds             y
0 2004-10-01  14284.521739
1 2004-10-02  12999.875000
2 2004-10-03  12227.083333
3 2004-10-04  14309.041667
4 2004-10-05  14439.708333
Daily data shape: (5055, 2)


In [24]:
# Initialize and train the model
model = Prophet()
model.fit(df_daily)

# Generate a future DataFrame for a 30-day forecast (or your desired horizon)
horizon = 30  # or another value
future = model.make_future_dataframe(periods=horizon)
forecast = model.predict(future)

print("Forecast preview:")
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())

14:25:58 - cmdstanpy - INFO - Chain [1] start processing
14:25:58 - cmdstanpy - INFO - Chain [1] done processing


Forecast preview:
             ds          yhat    yhat_lower    yhat_upper
5080 2018-08-29  15789.098078  14217.707249  17370.717843
5081 2018-08-30  15728.355636  14176.099175  17156.113632
5082 2018-08-31  15395.371679  13754.924703  16931.293107
5083 2018-09-01  14153.503991  12532.222487  15754.194970
5084 2018-09-02  13659.291423  12139.669240  15115.943776


In [26]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objects as go

# Initialize the Dash app (using JupyterDash if in a notebook)
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "Energy Consumption Forecast Dashboard"

# Define the app layout
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("Energy Consumption Forecast Dashboard"), width=12)
    ], className="my-2"),
    
    dbc.Row([
        dbc.Col([
            html.Label("Select Forecast Horizon (Days)"),
            dcc.Dropdown(
                id='horizon-dropdown',
                options=[
                    {'label': '30 Days', 'value': 30},
                    {'label': '90 Days', 'value': 90},
                    {'label': '365 Days (1 Year)', 'value': 365}
                ],
                value=30,
                clearable=False
            )
        ], width=3)
    ], className="my-2"),
    
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='forecast-graph')
        ], width=12)
    ]),
], fluid=True)

# Callback to update forecast graph based on selected horizon
@app.callback(
    Output('forecast-graph', 'figure'),
    Input('horizon-dropdown', 'value')
)
def update_forecast(horizon):
    # Train the Prophet model on the full daily data
    model = Prophet()
    model.fit(df_daily)
    
    # Create future dataframe and predict forecast
    future = model.make_future_dataframe(periods=horizon)
    forecast = model.predict(future)
    
    # Build the Plotly figure
    fig = go.Figure()
    
    # Historical data trace
    fig.add_trace(go.Scatter(
        x=df_daily['ds'],
        y=df_daily['y'],
        mode='lines',
        name='Historical Data'
    ))
    
    # Forecast trace
    fig.add_trace(go.Scatter(
        x=forecast['ds'],
        y=forecast['yhat'],
        mode='lines',
        name='Forecast',
        line=dict(color='red')
    ))
    
    # Confidence interval traces
    fig.add_trace(go.Scatter(
        x=forecast['ds'],
        y=forecast['yhat_lower'],
        fill=None,
        mode='lines',
        line=dict(color='lightpink'),
        name='Lower Confidence'
    ))
    fig.add_trace(go.Scatter(
        x=forecast['ds'],
        y=forecast['yhat_upper'],
        fill='tonexty',
        mode='lines',
        line=dict(color='lightpink'),
        name='Upper Confidence'
    ))
    
    fig.update_layout(
        title=f"Forecast for {horizon} Days",
        xaxis_title="Date",
        yaxis_title="Energy Consumption (MW)",
        hovermode="x unified"
    )
    return fig

# Run the app (if in a script, otherwise use app.run_server(mode='inline') for JupyterDash)
if __name__ == '__main__':
    app.run(debug=True)
